# Format datas

## Libs :

In [1]:
import json
import numpy as np
import pandas as pd

## Parameters :

In [2]:
path = 'output/romain'
rssi_csv_output = 'rssi.csv'

## Variables :

In [3]:
rssi = []
mac = []

## Importing datas :

In [4]:
# Opening JSON file
f = open('output/romain/h106.json')

# returns JSON object as a dictionary
data = json.load(f)

print("number of positions : ",len(data))
print("number of acquisitions (first acquisition) : ",len(data['output4']))
print("number of RSSI detected (first acquisition) : ",len(data['output4'][0]))
print("mac and RSSI (first acquisition) : ",len(data['output4'][0][0]))

#compute the number of acquisitions :
n_acquisition = 0
i_position = -1
for position in data:
    i_acquisition = -1
    i_position += 1
    for acquisition in data[position]:
        i_acquisition += 1
        n_access_points = len(data[position][i_acquisition])
        for i_access_point in range(n_access_points):
            i_mac = data[position][i_acquisition][i_access_point]['mac']
            #check if mac address is already known :
            if not i_mac in mac:
                mac.append(i_mac) #if not, add it in the mac array
    n_acquisition += i_acquisition + 1

n_mac = len(mac)
print("number of acquisition : ", n_acquisition)
print("number of access points (mac) detected : ", n_mac)

#initialize the array with all the rssi values :
rssi = np.zeros([n_acquisition, n_mac])

#print(data['output4'][1])

acquisition_index_for_rssi_array = -1
i_position = -1
for position in data:
    i_acquisition = -1
    i_position += 1
    #print("\n",i_position)
    for acquisition in data[position]:
        i_acquisition += 1
        acquisition_index_for_rssi_array += 1
        n_access_points = len(data[position][i_acquisition])
        #print("\n",i_acquisition)
        #print(n_access_points)
        #print(len(data[position][i_acquisition]))
        for i_access_point in range(n_access_points):
            #print("\n",i_access_point)
            i_mac = data[position][i_acquisition][i_access_point]['mac']
            #print("mac : ",mac)
            i_rssi = data[position][i_acquisition][i_access_point]['RSSI']
            #print("RSSI : ",rssi)
            #print(data[position][i_acquisition][i_access_point])
            #add the RSSI value in the rssi array :
            #print(mac.index(i_mac))
            rssi[acquisition_index_for_rssi_array, mac.index(i_mac)] = i_rssi

print(rssi)

# convert array into dataframe
DF = pd.DataFrame(rssi)
  
# save the dataframe as a csv file
DF.to_csv(rssi_csv_output)

number of positions :  11
number of acquisitions (first acquisition) :  5
number of RSSI detected (first acquisition) :  49
mac and RSSI (first acquisition) :  2
number of acquisition :  55
number of access points (mac) detected :  154
[[-66. -66. -65. ...   0.   0.   0.]
 [-68. -65. -65. ...   0.   0.   0.]
 [-68. -65. -66. ...   0.   0.   0.]
 ...
 [-60. -73. -72. ... -83. -83.   0.]
 [-56. -72. -73. ... -83. -83.   0.]
 [-55. -71. -70. ... -83. -83. -83.]]
